In [1]:
from bokeh.plotting import show, figure
from bokeh.io import output_notebook 
import pickle as pk
import numpy as np

## Load inline plotting 

In [2]:
output_notebook()

Loading BokehJS ...

## Unpickle data

In [3]:
result_dict = pk.load(open('example_output.p','r'))

## Print out all elements in dictionary 
- See Wiki for explanation of all outputs  

In [4]:
print result_dict.keys()
print result_dict['FinalSpectrum'].keys()

['FinalSpectrum', 'OriginalInput', 'timing_div', 'warnings_div', 'PandeiaOutTrans', 'input_div', 'warning', 'RawData', 'timing', 'input']
['spectrum_w_rand', 'spectrum', 'error_w_floor', 'wave']


## 1D Model + Data + Error bars 

In [18]:
#remove any nans 
y = result_dict['FinalSpectrum']['spectrum_w_rand']
x = result_dict['FinalSpectrum']['wave'][~np.isnan(y)]
err = result_dict['FinalSpectrum']['error_w_floor'][~np.isnan(y)]
y = y[~np.isnan(y)]

#create error bars for Bokeh's multi_line
y_err = []
x_err = []
for px, py, yerr in zip(x, y, err):
    np.array(x_err.append((px, px)))
    np.array(y_err.append((py - yerr, py + yerr)))
ylims = [min(result_dict['OriginalInput']['og_spec'])- 0.1*min(result_dict['OriginalInput']['og_spec']),
                 0.1*max(result_dict['OriginalInput']['og_spec'])+max(result_dict['OriginalInput']['og_spec'])]
xlims = [min(x), max(x)]
#initialize figure 
fig1d = figure(x_range=xlims, y_range = ylims, 
               plot_width = 500, plot_height =300,title='Model+Data+Error',x_axis_label='Wavelength (um)',
              y_axis_label = '(Rp/R*)^2')
#plot model, data, and errors 
fig1d.line(result_dict['OriginalInput']['og_wave'], result_dict['OriginalInput']['og_spec'], color='black',alpha=0.5)
fig1d.circle(x, y)
fig1d.multi_line(x_err, y_err)
show(fig1d)

## 1D Extracted Flux 

In [16]:
#out of transit 1d output 
out = result_dict['PandeiaOutTrans']

# Flux 1d
x, y = out['1d']['extracted_flux']
x = x[~np.isnan(y)]
y = y[~np.isnan(y)]

plot_flux_1d1 = figure(x_axis_label='Wavelength [microns]',
                     y_axis_label='Flux (e/s)', title="Out of Transit Flux",
                     plot_width=500, plot_height=300)
plot_flux_1d1.line(x, y, line_width = 4, alpha = .7)
show(plot_flux_1d1)

## 1D SNR Out of Transit

In [23]:
#out of transit 1d output 
out = result_dict['PandeiaOutTrans']
# SNR 1d accounting for number of occultations
x, y = out['1d']['sn']
x = x[~np.isnan(y)]
y = y[~np.isnan(y)]
#pull out number of transits 
noccultations = result_dict['timing']['Number of Transits']
#increase snr by number transits
y = y*np.sqrt(noccultations)
#initialize plot
plot_snr_1d1 = figure(x_axis_label='Wavelength [microns]',
                     y_axis_label='SNR', title="Signal-to-Noise Ratio",
                     plot_width=500, plot_height=300)
plot_snr_1d1.line(x, y, line_width = 4, alpha = .7)
show(plot_snr_1d1)

## 1D Background Out of Transit

In [28]:
#out of transit 1d output 
out = result_dict['PandeiaOutTrans']

x, y = out['1d']['bg']
y = y[~np.isnan(y)]
x = x[~np.isnan(y)]
plot_bg_1d1 = figure(x_axis_label='Wavelength [microns]',
                     y_axis_label='Flux (e/s)', title="Background",
                     plot_width=500, plot_height=300)
plot_bg_1d1.line(x, y, line_width = 4, alpha = .7)
show(plot_bg_1d1)